# More isochrones with OAlley


[OAlley API](https://api.oalley.fr) lets you compute [isochrones](https://en.wikipedia.org/wiki/Isochrone_map).

In this example, we use the `grequest` module to request multiple isochrones from the server in parallel. This is useful if you need a lot of them quickly.


## Running this example

If you want to run this example to compute your own isochrones, follow the [installation instructions](https://github.com/Overdrivr/interactive-geospatial-analysis).

Also, you will need an OAlley API key. Go to [api.oalley.fr](https://api.oalley.fr) and signup. It's free. Each new user receives a key with 1000 requests available.

Paste the key in the code below.

In [2]:
import folium
import grequests
import polyline
from urllib.parse import urlencode

baseurl = "https://api.oalley.fr/api/AppKeys/"
method = "/isochronous?"

appkey = "YOUR-API-KEY"


## Prepare the requests

We are going to compute isochronous areas of 1min, 2min and 3min by car around the [Arc de Triomphe](http://tools.wmflabs.org/geohack/geohack.php?language=fr&pagename=Arc_de_triomphe_de_l%27%C3%89toile&params=48.8738_N_2.295_E_) in Paris.

In [4]:
points = [
    {
        "lat":48.8738,
        "lng":2.295,
        "duration": 60 # 1 min by car
    },
    {
        "lat":48.8738,
        "lng":2.295,
        "duration":120 # 2 min
    },
    {
        "lat":48.8738,
        "lng":2.295,
        "duration":180 # 3 min
    }    
]

urls = [baseurl + appkey + method + urlencode(point) for point in points]

urls

['https://api.oalley.fr/api/AppKeys/YOUR-API-KEY/isochronous?lng=2.295&duration=60&lat=48.8738',
 'https://api.oalley.fr/api/AppKeys/YOUR-API-KEY/isochronous?lng=2.295&duration=120&lat=48.8738',
 'https://api.oalley.fr/api/AppKeys/YOUR-API-KEY/isochronous?lng=2.295&duration=180&lat=48.8738']

## Call the API

The `callback` function is executed every time a request has responded. We use it to convert the zone from the standard [`polyline`](https://developers.google.com/maps/documentation/utilities/polylinealgorithm) format to the more user-friendly JSON format.

In [3]:
zones = []

# Called when OAlley returned an isochrone
def callback(res, **kwargs):
    if(res.status_code != 200): 
        return print(res.json())
    
    body = res.json()
    zone = polyline.decode(body['polyline'])
    zones.append(zone)
    
def exception_handler(request, exception):
    print("Error : %s" % exception)
    
# Prepare all requests
reqs = [grequests.get(url, callback=callback) for url in urls]

# Execute all requests
grequests.map(reqs, exception_handler=exception_handler)

[<Response [200]>, <Response [200]>, <Response [200]>]

## Print the results

We will now use a folium map to print the isochronous areas on a map.

In [5]:
# Build output map
m = folium.Map(location=[46, 2], zoom_start=5)

# Draw all computed zones on the map
for points in zones:
    # Trick to close the polyline, until folium library supports it
    points.append(points[0])
    p = folium.PolyLine(locations=points,weight=3)
    m.add_children(p)

# Print the result
m.fit_bounds(m.get_bounds())

# IPython trick to display the map
m

## What's next

Go to the [`repository page`](https://github.com/Overdrivr/interactive-geospatial-analysis) to see all other examples and experimentations, test them yourself and contribute if you feel like it.


![compute-all.jpg](./compute-all-iso.jpg)